In [ ]:
%cd /content/drive/MyDrive/TARP/data

/content/drive/MyDrive/TARP/data


In [ ]:
import torchvision

In [ ]:
torch.__version__

'1.11.0+cu113'

In [ ]:
import cv2
import numpy as np
from numpy.core.defchararray import join, mod
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
import os
import torch
from torch._C import device
from torch.autograd.grad_mode import F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
from torchvision import transforms as torchtrans  

In [ ]:
labels_path = 'Labels'
imgs_path = 'Images'

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

class gun(Dataset):
    def __init__(self,imgs_path,labels_path):

        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.img_name = [img for img in sorted(os.listdir(self.imgs_path))]
        self.label_name = [label for label in sorted(os.listdir(self.labels_path))]

    def __getitem__(self,idx):

        image_path = os.path.join(self.imgs_path,str(self.img_name[idx]))
        img = cv2.imread(image_path)
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = img_rgb/255
        img_res = torch.as_tensor(img_res).to(device)
        img_res = img_res.permute(2, 0, 1)
        
        label_name = self.img_name[idx][:-4] + "txt"
        label_path = os.path.join(self.labels_path,str(label_name))
        with open(label_path, 'r') as label_file:
            l_count = int(label_file.readline())
            box = []
            for i in range(l_count):
                box.append(list(map(int, label_file.readline().split())))

        target={}
        target["boxes"] = torch.as_tensor(box).to(device)
        area = []
        for i in range(len(box)):
           
            a = (box[i][2] - box[i][0]) * (box[i][3] - box[i][1])
            area.append(a)
        target["area"] = torch.as_tensor(area).to(device)
        labels = []
        for i in range(len(box)):
            labels.append(1)

        target["image_id"] = torch.as_tensor([idx]).to(device)
        target["labels"] = torch.as_tensor(labels, dtype = torch.int64).to(device)


        return img_res,target

    def __len__(self):
        return len(self.img_name)

In [ ]:
def model(num):
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(
        in_features, num)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
gun_data = gun(imgs_path, labels_path)
gun_data

In [ ]:
data_load = DataLoader(gun_data, batch_size=5,
                       shuffle=True, num_workers=0, collate_fn=collate_fn)
num_class = 2
model = model(num_class)
model.to(device)
num_epoch = 10
param = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.SGD(param,lr=0.01,momentum=0.9)
lr_scheduler = None
for epoch in range(num_epoch):
    loss_num = 0
    model.train()
    for img, target in data_load:
        loss_dic = model(img, target)
        loss = sum(loss for loss in loss_dic.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_num += loss
    if lr_scheduler is not None:
        lr_scheduler.step()
    print("epoch:{},loss:{}".format(epoch, loss_num))

In [ ]:
cpu_device = torch.device("cpu")
model = model.eval()
test_path = 'Images'
test_data = gun(imgs_path, labels_path)
img,tar = test_data[10] 
input = []
input.append(img)
outputs = model(input)

In [ ]:
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
boxes = outputs[0]["boxes"].detach().numpy().astype(np.int32)
id = np.argmax(outputs[0]["scores"].detach().numpy())
xmin = boxes[id][0]
xmax = boxes[id][2]
ymin = boxes[id][1]
ymax = boxes[id][3]
img1 = np.array(img.permute(1, 2, 0).to(torch.device("cpu")))

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(img1,(xmin,ymin),(xmax,ymax),(220,0,0),1)
    
ax.set_axis_off()
ax.imshow(img1)
outputs 

In [ ]:
import matplotlib.patches as patches
def apply_nms(orig_prediction, iou_thresh=None):
    
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]

    
    return final_prediction

In [ ]:
def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')


def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

In [ ]:
nms_prediction = apply_nms(outputs[0], iou_thresh=0.7)
print('NMS APPLIED MODEL OUTPUT')
plot_img_bbox(torch_to_pil(img), nms_prediction)